In [2]:
from agent import getState, getStockDataVec, Agent, formatPrice
from keras.models import load_model

In [3]:
# Arguments

stock_name = "VOO"
window_size = 10
episode_count = 10

Sell: Rs.405.58 | Profit: Rs.1.97
Buy: Rs.403.64
Sell: Rs.406.16 | Profit: Rs.2.77
Sell: Rs.406.86 | Profit: Rs.3.22
Buy: Rs.406.47
Sell: Rs.406.98 | Profit: Rs.0.51
Buy: Rs.408.23
Buy: Rs.403.84
Buy: Rs.407.61
Sell: Rs.411.22 | Profit: Rs.2.99
Buy: Rs.411.87
Buy: Rs.410.22
Buy: Rs.413.86
Buy: Rs.415.76
Buy: Rs.415.05
Sell: Rs.415.49 | Profit: Rs.11.65
Buy: Rs.416.73
Sell: Rs.416.57 | Profit: Rs.8.96
Sell: Rs.414.63 | Profit: Rs.2.76
Sell: Rs.412.77 | Profit: Rs.2.55
Sell: Rs.410.54 | Profit: -Rs.3.32
Sell: Rs.408.41 | Profit: -Rs.7.35
Sell: Rs.411.82 | Profit: -Rs.3.23
Sell: Rs.411.11 | Profit: -Rs.5.62
Buy: Rs.408.78
Sell: Rs.409.46 | Profit: Rs.0.68
Buy: Rs.408.30
Sell: Rs.400.14 | Profit: -Rs.8.16
Buy: Rs.398.04
Sell: Rs.399.81 | Profit: Rs.1.77
Buy: Rs.399.56
Sell: Rs.398.76 | Profit: -Rs.0.80
Buy: Rs.411.18
Buy: Rs.414.38
Buy: Rs.415.99
Buy: Rs.417.01
Buy: Rs.416.59
Sell: Rs.418.75 | Profit: Rs.7.57
Sell: Rs.417.33 | Profit: Rs.2.95
Sell: Rs.421.41 | Profit: Rs.5.42
Sell: Rs.422.

KeyboardInterrupt: 

In [ ]:
# Train
agent = Agent(window_size)
data = getStockDataVec(stock_name)
l = len(data) - 1
batch_size = 32
for e in range(episode_count + 1):
    print("Episode " + str(e) + "/" + str(episode_count))
    state = getState(data, 0, window_size + 1)
    total_profit = 0
    agent.inventory = []
    for t in range(l):
        action = agent.act(state)
        # sit
        next_state = getState(data, t + 1, window_size + 1)
        reward = 0
        if action == 1:  # buy
            agent.inventory.append(data[t])
            print("Buy: " + formatPrice(data[t]))
        elif action == 2 and len(agent.inventory) > 0:  # sell
            bought_price = window_size_price = agent.inventory.pop(0)
            reward = max(data[t] - bought_price, 0)
            total_profit += data[t] - bought_price
            print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))
        done = True if t == l - 1 else False
        agent.memory.append((state, action, reward, next_state, done))
        state = next_state
        if done:
            print("--------------------------------")
            print("Total Profit: " + formatPrice(total_profit))
            print("--------------------------------")
        if len(agent.memory) > batch_size:
            agent.expReplay(batch_size)
    if e % 10 == 0:
        agent.model.save(str(e))

/usr/local/lib/python3.9/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Episode 0/10
Buy: Rs.340.16
Sell: Rs.337.14 | Profit: -Rs.3.02
Buy: Rs.335.01
Buy: Rs.339.64
Buy: Rs.340.16
Buy: Rs.342.01
Buy: Rs.340.64
Buy: Rs.339.59
Sell: Rs.337.81 | Profit: Rs.2.80
Sell: Rs.339.16 | Profit: -Rs.0.48
Sell: Rs.342.11 | Profit: Rs.1.95
Sell: Rs.341.85 | Profit: -Rs.0.16
Buy: Rs.343.69
Sell: Rs.339.03 | Profit: -Rs.1.61
Buy: Rs.343.33
Sell: Rs.350.42 | Profit: Rs.10.83
Buy: Rs.348.13
Buy: Rs.348.16
Buy: Rs.347.85
Sell: Rs.345.37 | Profit: Rs.1.68
Buy: Rs.352.83
Buy: Rs.353.33
Sell: Rs.352.80 | Profit: Rs.9.47
Sell: Rs.340.18 | Profit: -Rs.7.95
Sell: Rs.345.77 | Profit: -Rs.2.39
Buy: Rs.351.09
Buy: Rs.355.02
Buy: Rs.356.44
Sell: Rs.358.78 | Profit: Rs.10.93
Sell: Rs.359.22 | Profit: Rs.6.39
Buy: Rs.361.05
Buy: Rs.360.79
Sell: Rs.359.25 | Profit: Rs.5.92
Sell: Rs.351.52 | Profit: Rs.0.43
Sell: Rs.349.59 | Profit: -Rs.5.43
Buy: Rs.355.35
Sell: Rs.350.66 | Profit: -Rs.5.78
Sell: Rs.346.34 | Profit: -Rs.14.71
Sell: Rs.350.96 | Profit: -Rs.9.83
Buy: Rs.358.07
Buy: Rs.362.3

In [ ]:
# Eval

model_name = input()
model = load_model(model_name)
window_size = model.layers[0].input.shape.as_list()[1]
agent = Agent(window_size, True, model_name)
data = getStockDataVec(stock_name)
print(data)
l = len(data) - 1
batch_size = 32
state = getState(data, 0, window_size + 1)
print(state)
total_profit = 0
agent.inventory = []
print(l)
for t in range(l):
    action = agent.act(state)
    print(action)
    # sit
    next_state = getState(data, t + 1, window_size + 1)
    reward = 0
    if action == 1:  # buy
        agent.inventory.append(data[t])
        print("Buy: " + formatPrice(data[t]))
    elif action == 2 and len(agent.inventory) > 0:  # sell
        bought_price = agent.inventory.pop(0)
        reward = max(data[t] - bought_price, 0)
        total_profit += data[t] - bought_price
        print("Sell: " + formatPrice(data[t]) + " | Profit: " + formatPrice(data[t] - bought_price))
    done = True if t == l - 1 else False
    agent.memory.append((state, action, reward, next_state, done))
    state = next_state
    if done:
        print("--------------------------------")
        print(stock_name + " Total Profit: " + formatPrice(total_profit))
        print("--------------------------------")
        print("Total profit is:", formatPrice(total_profit))